In [ ]:
# STEP 1: INSTALL DEPENDENCIES
!pip install -qq pyannote.audio==3.1.1 noisereduce librosa ffmpeg-python faster-whisper
!apt-get install -qq ffmpeg
!pip install "numpy<2.0"

# STEP 2: SETUP HUGGINGFACE TOKEN (for pyannote)
from getpass import getpass

HUGGINGFACE_TOKEN = getpass("Enter your HuggingFace token:")
from huggingface_hub import login
login(HUGGINGFACE_TOKEN)

print("Dependencies installed and HuggingFace token set up.")

Enter your HuggingFace token:··········
Dependencies installed and HuggingFace token set up.


In [ ]:
# STEP 3: UPLOAD AUDIO FILE
from google.colab import files
import os

uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Convert to WAV using ffmpeg
import ffmpeg

base_wav = "converted_audio.wav"
ffmpeg.input(filename).output(base_wav, ac=1, ar='16000').overwrite_output().run()
print(f"Audio converted to {base_wav}")

Saving Card Not Deliverd And Activation Issues.m4a to Card Not Deliverd And Activation Issues.m4a
Audio converted to converted_audio.wav


In [ ]:
# STEP 4: NOISE REDUCTION
import librosa
import noisereduce as nr
import soundfile as sf

y, sr = librosa.load(base_wav, sr=16000)
noise_sample = y[:int(sr * 0.5)]
reduced_noise = nr.reduce_noise(y=y, sr=sr, y_noise=noise_sample)
sf.write(base_wav, reduced_noise, sr)
print("Noise reduction complete.")

Noise reduction complete.


In [ ]:
# STEP 5: SPEAKER DIARIZATION
from pyannote.audio import Pipeline
from datetime import timedelta

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HUGGINGFACE_TOKEN
)

diarization = pipeline(base_wav)
rttm_path = "diarization.rttm"
with open(rttm_path, "w") as f:
    diarization.write_rttm(f)
print("Diarization complete. RTTM saved.")


/usr/local/lib/python3.11/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
/usr/local/lib/python3.11/dist-packages/pyannote/audio/pipelines/speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your scr

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


Diarization complete. RTTM saved.


In [ ]:
# STEP 6: TRANSCRIPTION USING WHISPER
from faster_whisper import WhisperModel

model_size = "medium"  # Or "small", "large-v2"
# Changed device to "cpu" to avoid CUDA compatibility issues
whisper_model = WhisperModel(model_size, device="cpu", compute_type="int8")

# Extract diarization segments
segments = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    start = turn.start
    end = turn.end
    if end - start >= 0.5:
        segments.append({
            "speaker": speaker,
            "start": start,
            "end": end
        })

# Transcribe each segment
import tempfile
import json

results = []
for i, seg in enumerate(segments):
    start, end = seg["start"], seg["end"]
    speaker = seg["speaker"]

    segment_path = f"segment_{i}.wav"
    # Ensure ffmpeg is imported if this cell is run independently
    try:
        ffmpeg.input(base_wav, ss=start, to=end).output(segment_path, ac=1, ar='16000').overwrite_output().run()
    except NameError:
        import ffmpeg
        ffmpeg.input(base_wav, ss=start, to=end).output(segment_path, ac=1, ar='16000').overwrite_output().run()


    # Transcribe
    transcription, _ = whisper_model.transcribe(segment_path, beam_size=5)
    text = " ".join([seg.text for seg in transcription])

    results.append({
        "speaker": speaker,
        "start": float(start),
        "end": float(end),
        "transcript": text
    })

with open("diarized_transcription.json", "w") as f:
    json.dump(results, f, indent=4)

print("Transcription complete. Output saved to diarized_transcription.json")
print("Transcription completed. First 5 results:")
print("Full Diarized Transcription (one per line):")
for entry in results:
    print(f"Speaker {entry['speaker']} [{entry['start']:.2f} - {entry['end']:.2f}]:\n{entry['transcript']}\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

Transcription complete. Output saved to diarized_transcription.json
Transcription completed. First 5 results:
Full Diarized Transcription (one per line):
Speaker SPEAKER_01 [1.10 - 1.79]:
 Good afternoon.

Speaker SPEAKER_01 [1.99 - 7.21]:
 Thank you for calling SBI Bank customer support.  My name is Gautam. How may I assist you today?

Speaker SPEAKER_00 [8.14 - 13.93]:
 Hi Gautam, I applied for a debit card around 2 weeks ago but I still haven't received it.

Speaker SPEAKER_01 [14.64 - 21.32]:
 I am sorry to hear them.  Let me check the status for you.  For verification could you please share your digital mobile number and

Speaker SPEAKER_01 [21.33 - 22.15]:
 The date of birth.

Speaker SPEAKER_00 [22.89 - 31.67]:
 Sure, my number is 98765-43210 and my date of birth is 15th August 2001.

Speaker SPEAKER_01 [32.39 - 35.19]:
 Thank you. Just a moment while I pull up your details.

Speaker SPEAKER_01 [40.82 - 49.80]:
 Alright, I see that your car was dispatched on June 3rd via blue do

In [ ]:
# STEP 7: CLEAN-UP AND DOWNLOAD FILES
from google.colab import files
import shutil
import glob
import os

# Download results
files.download("diarized_transcription.json")
files.download("diarization.rttm")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -qq transformers sentencepiece

In [ ]:
from transformers import pipeline

# Load a summarization pipeline (you can use 'facebook/bart-large-cnn', 'google/pegasus-cnn_dailymail', etc.)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Concatenate all transcripts
full_text = " ".join([seg["transcript"] for seg in results if seg["transcript"]])

# Optionally truncate to first N chars if extremely long (Hugging Face models usually have max token limits)
MAX_CHARS = 3000
text_for_summary = full_text[:MAX_CHARS]

# Generate summary
summary = summarizer(text_for_summary, max_length=200, min_length=50, do_sample=False)[0]['summary_text']

# Print and save
print("\n📋 Call Summary:")
print(summary)

with open("call_summary.txt", "w") as f:
    f.write(summary)

from google.colab import files
files.download("call_summary.txt")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



📋 Call Summary:
Gautam applied for a debit card around 2 weeks ago but still haven't received it. Gautam, please share your digital mobile number and date of birth. I will do the delivery investigation and initiate a free reshift of your card. You will receive your new card within 5 to 7 working days.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Clean up
if os.path.exists(base_wav):
    os.remove(base_wav)
if os.path.exists("diarization.rttm"):
    os.remove("diarization.rttm")
if os.path.exists("diarized_transcription.json"):
    os.remove("diarized_transcription.json")


for seg_file in glob.glob("segment_*.wav"):
    os.remove(seg_file)

print("Temporary files removed. All done!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Temporary files removed. All done!
